In [1]:

import numpy as np
import torch 
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import gym
import gym_2048
from tqdm import tqdm

In [2]:
env = gym.make('2048-v0')

c:\Users\iguti\.virtualenvs\yannik_scribble-sRH0UPe6\lib\site-packages\gym\utils\passive_env_checker.py:20: UserWarning: WARN: It seems a Box observation space is an image but the `dtype` is not `np.uint8`, actual type: int32. If the Box observation space is not an image, we recommend flattening the observation to have only a 1D vector.
  logger.warn(
c:\Users\iguti\.virtualenvs\yannik_scribble-sRH0UPe6\lib\site-packages\gym\utils\passive_env_checker.py:25: UserWarning: WARN: It seems a Box observation space is an image but the upper and lower bounds are not in [0, 255]. Generally, CNN policies assume observations are within that range, so you may encounter an issue if the observation values are not.
  logger.warn(


In [3]:
replay_size = 10000

In [4]:
# MLP UDPRL stolen from https://github.com/BY571/Upside-Down-Reinforcement-Learning/blob/master/Upside-Down.ipynb

class BF(nn.Module):
    def __init__(self, state_space, action_space, hidden_size, seed):
        super(BF, self).__init__()
        torch.manual_seed(seed)
        self.actions = np.arange(action_space)
        self.action_space = action_space
        self.fc1 = nn.Linear(state_space, hidden_size)
        self.commands = nn.Linear(2, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, hidden_size)
        self.fc4 = nn.Linear(hidden_size, hidden_size)
        self.fc5 = nn.Linear(hidden_size, hidden_size)
        self.fc6 = nn.Linear(hidden_size, hidden_size)
        self.fc7 = nn.Linear(hidden_size, hidden_size)
        self.fc8 = nn.Linear(hidden_size, hidden_size)
        self.fc9 = nn.Linear(hidden_size, action_space)

        self.sigmoid = nn.Sigmoid()
        
    def forward(self, state, command):       
               
        out = self.sigmoid(self.fc1(state))
        command_out = self.sigmoid(self.commands(command))
        out = out * command_out
        out = torch.relu(self.fc2(out))
        out = torch.relu(self.fc3(out))
        out = torch.relu(self.fc4(out))
        out = torch.relu(self.fc5(out))
        out = torch.relu(self.fc6(out))
        out = torch.relu(self.fc7(out))
        out = torch.relu(self.fc8(out))
        out = self.fc9(out)
        
        return out
    
    def action(self, state, desire, horizon):
        """
        Samples the action based on their probability
        """
        command = torch.cat((desire,horizon), dim=-1)
        action_prob = self.forward(state.expand(1, -1), command.expand(1, -1))[0,:]
        probs = torch.softmax(action_prob, dim=-1)
        action = torch.distributions.categorical.Categorical(probs=probs).sample()
        return action
    def greedy_action(self, state, desire, horizon):
        """
        Returns the greedy action 
        """
        command = torch.cat((desire,horizon), dim=-1)
        action_prob = self.forward(state, command)
        probs = torch.softmax(action_prob, dim=-1)
        action = torch.argmax(probs).item()
        return action

In [5]:
import random


class ReplayBuffer():
    def __init__(self, max_size):
        self.max_size = max_size
        self.buffer = []
        
        
    def add_sample(self, states, actions, rewards):
        episode = {"states": states, "actions":actions, "rewards": rewards, "summed_rewards":sum(rewards)}
        self.buffer.append(episode)

    def get_nbest(self, n):
        self.sort()
        return self.buffer[:n]
        
    
    def sort(self):
        #sort buffer
        self.buffer = sorted(self.buffer, key = lambda i: i["summed_rewards"],reverse=True)
        # keep the max buffer size
        self.buffer = self.buffer[:self.max_size]
    
    def get_random_samples(self, batch_size):
        self.sort()
        idxs = np.random.randint(0, len(self.buffer), batch_size)
        batch = [self.buffer[idx] for idx in idxs]
        return batch
    
    def __len__(self):
        return len(self.buffer)

In [6]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

bf = BF(256, 4, 2048, 1).to(device)
optimizer = optim.Adam(params=bf.parameters(), lr=1e-3)

In [7]:
init_desired_reward = 1
init_time_horizon = 2

def generate_episode(desired_return = torch.FloatTensor([init_desired_reward]), desired_time_horizon = torch.FloatTensor([init_time_horizon])):    
    """
    Generates more samples for the replay buffer.
    """
    next_state = env.reset()
    states = []
    actions = []
    rewards = []
    done = False
    while not done:
        action = bf.action(torch.from_numpy(next_state.flatten()).float().to(device), desired_return, desired_time_horizon)
        next_state, reward, done, info = env.step(action)
        states.append(next_state.flatten())
        desired_return -= reward
        desired_time_horizon -= 1
        desired_time_horizon = torch.FloatTensor([np.maximum(desired_time_horizon, 1).item()])
        actions.append(action)
        rewards.append(reward)
    return states, actions, rewards, info['highest']

In [8]:
# create replay buffer:
buffer = ReplayBuffer(replay_size)
ma = 0
# init replay buffer with random trajectories:
for i in tqdm(range(0, int(replay_size/10))):
    states, actions, rewards, h = generate_episode()
    ma = max(h, ma)
    buffer.add_sample(states, actions, rewards)
print("Max: ", ma)

  0%|          | 0/1000 [00:00<?, ?it/s]c:\Users\iguti\.virtualenvs\yannik_scribble-sRH0UPe6\lib\site-packages\gym\utils\passive_env_checker.py:174: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed a `seed` instead of using `Env.seed` for resetting the environment random number generator.
  logger.warn(
c:\Users\iguti\.virtualenvs\yannik_scribble-sRH0UPe6\lib\site-packages\gym\utils\passive_env_checker.py:187: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed `options` to allow the environment initialisation to be passed additional information.
  logger.warn(
c:\Users\iguti\.virtualenvs\yannik_scribble-sRH0UPe6\lib\site-packages\gym\utils\passive_env_checker.py:195: UserWarning: WARN: The result returned by `env.reset()` was not a tuple of the form `(obs, info)`, where `obs` is a observation and `info` is a dictionary containing additional information. Actual type: `<class 'numpy.ndarray'>`
  logger.warn(
c:\Users\iguti

Max:  64


In [9]:
# FUNCTIONS FOR Sampling exploration commands
last_few = 1000
def sampling_exploration( top_X_eps = last_few):
    """
    This function calculates the new desired reward and new desired horizon based on the replay buffer.
    New desired horizon is calculted by the mean length of the best last X episodes. 
    New desired reward is sampled from a uniform distribution given the mean and the std calculated from the last best X performances.
    where X is the hyperparameter last_few.
    
    """
    
    top_X = buffer.get_nbest(last_few)
    #The exploratory desired horizon dh0 is set to the mean of the lengths of the selected episodes
    new_desired_horizon = np.mean([len(i["states"]) for i in top_X])
    # save all top_X cumulative returns in a list 
    returns = [i["summed_rewards"] for i in top_X]
    # from these returns calc the mean and std
    mean_returns = np.mean(returns)
    std_returns = np.std(returns)
    # sample desired reward from a uniform distribution given the mean and the std
    new_desired_reward = np.random.uniform(mean_returns, mean_returns+std_returns)

    return torch.FloatTensor([new_desired_reward])  , torch.FloatTensor([new_desired_horizon]) 

In [10]:


# FUNCTIONS FOR TRAINING
def select_time_steps(saved_episode):
    """
    Given a saved episode from the replay buffer this function samples random time steps (t1 and t2) in that episode:
    T = max time horizon in that episode
    Returns t1, t2 and T 
    """
    # Select times in the episode:
    T = len(saved_episode["states"]) # episode max horizon
    if T != 1:
        t1 = np.random.randint(0,T-1)
        t2 = np.random.randint(t1+1,T)
    else:
        t1 = 0
        t2 = T

    return t1, t2, T

def create_training_input(episode, to):
    """
    Based on the selected episode and the given time steps this function returns 4 values:
    1. state at t1
    2. the desired reward: sum over all rewards from t1 to t2
    3. the time horizont: t2 -t1
    
    4. the target action taken at t1
    
    buffer episodes are build like [cumulative episode reward, states, actions, rewards]
    """
    state = episode["states"][to] 
    rtg = sum(episode["rewards"])
    rtg = rtg - episode['rewards']
    timesteps = t2-t1
    action = episode["actions"][t1]
    return state, desired_reward, time_horizont, action

def create_training_examples(batch_size):
    """
    Creates a data set of training examples that can be used to create a data loader for training.
    ============================================================
    1. for the given batch_size episode idx are randomly selected
    2. based on these episodes t1 and t2 are samples for each selected episode 
    3. for the selected episode and sampled t1 and t2 trainings values are gathered
    ______________________________________________________________
    Output are two numpy arrays in the length of batch size:
    Input Array for the Behavior function - consisting of (state, desired_reward, time_horizon)
    Output Array with the taken actions 
    """
    input_array = []
    output_array = []
    # select randomly episodes from the buffer
    episodes = buffer.get_random_samples(batch_size)
    for ep in episodes:
        #select time stamps
        t1, t2, T = select_time_steps(ep)
        # For episodic tasks they set t2 to T:
        t2 = T
        state, desired_reward, time_horizont, action = create_training_input(ep, t1, t2)
        input_array.append(torch.cat([torch.from_numpy(state).float(), torch.FloatTensor([desired_reward]), torch.FloatTensor([time_horizont])]))
        output_array.append(action)
    return input_array, output_array

def train_behavior_function(batch_size):
    """
    Trains the BF with on a cross entropy loss were the inputs are the action probabilities based on the state and command.
    The targets are the actions appropriate to the states from the replay buffer.
    """
    X, y = create_training_examples(batch_size)

    X = torch.stack(X)
    state = X[:,0:256]
    d = X[:,256:256+1]
    h = X[:,256+1:256+2]
    command = torch.cat((d,h), dim=-1)
    y = torch.stack(y).long()
    y_ = bf(state.to(device), command.to(device)).float()
    optimizer.zero_grad()
    pred_loss = F.cross_entropy(y_, y)   
    pred_loss.backward()
    optimizer.step()
    return pred_loss.detach().cpu().numpy()



In [15]:
def run_loop():
    i = 0
    while True:
        # compute desired horizon and desired reward
        
        i += 1
        ma = 0
        for _ in range(0, int(replay_size/10000)):
            rew, hor = sampling_exploration()
            states, actions, rewards, h = generate_episode(rew, hor)
            ma = max(h,ma)
            buffer.add_sample(states, actions, rewards)

        loss = train_behavior_function(int(replay_size*10))
        print(i, loss, "Max: ", ma, " RewardAim: ", rew, " HorizonAim: ", hor)

In [16]:
run_loop()

1 1.3893855 Max:  16  RewardAim:  tensor([258.4325])  HorizonAim:  tensor([43.2080])
2 1.3744199 Max:  32  RewardAim:  tensor([81.3916])  HorizonAim:  tensor([43.2080])
3 1.3905137 Max:  4  RewardAim:  tensor([319.1218])  HorizonAim:  tensor([43.2110])
4 1.3844566 Max:  16  RewardAim:  tensor([239.0560])  HorizonAim:  tensor([43.2110])
5 1.3768514 Max:  4  RewardAim:  tensor([325.5722])  HorizonAim:  tensor([43.2110])
6 1.377079 Max:  8  RewardAim:  tensor([285.9692])  HorizonAim:  tensor([43.2110])
7 1.3841692 Max:  16  RewardAim:  tensor([186.6310])  HorizonAim:  tensor([43.2110])
8 1.3865682 Max:  8  RewardAim:  tensor([324.3599])  HorizonAim:  tensor([43.2110])
9 1.380036 Max:  16  RewardAim:  tensor([230.4178])  HorizonAim:  tensor([43.2110])
10 1.3765268 Max:  4  RewardAim:  tensor([308.7067])  HorizonAim:  tensor([43.2110])
11 1.3735746 Max:  4  RewardAim:  tensor([369.4398])  HorizonAim:  tensor([43.2110])
12 1.3692938 Max:  8  RewardAim:  tensor([305.0653])  HorizonAim:  tenso